In [1]:
import pandas as pd
import numpy as np
import numpy
import new_extractor as ext
import os

In [2]:
def readExecutionData(num_experiences, framework, device, benchmark, benchmark_path):
    executions = []
    for execution_counter in range(1, num_experiences + 1):
        memory_path = os.path.join(benchmark_path, f"meminfo-{execution_counter}.txt")
        try:
            memory_data = ext.get_memory(memory_path)
        except FileNotFoundError:
            memory_data = [numpy.NaN] * 4

        energy_path = os.path.join(benchmark_path, f"batterystats-{execution_counter}.txt")
        try:
            voltage = ext.get_voltage(energy_path)
            app_id = ext.get_appid(energy_path)

            energy = ext.get_energy(energy_path, app_id, voltage)
            time_data = ext.get_time(energy_path, app_id)
        except FileNotFoundError:
            energy = numpy.NaN
            time_data = [numpy.NaN] * 2
        
        data_path = os.path.join(benchmark_path, f"procstats-{execution_counter}.txt")
        try:
            package = ext.get_package(energy_path)
            proc_data = ext.get_data(data_path, package)
        except FileNotFoundError:
            proc_data = [numpy.NaN] * 3
            package = None

        frame_path = os.path.join(benchmark_path, f"gfxinfo-{execution_counter}.txt")
        try:
            pass
        except FileNotFoundError:
            pass
        
        executions.append([framework, device, benchmark, execution_counter, *memory_data, *proc_data, energy, *time_data])
    return executions

def getDataFrame(benchmarks):
    return pd.DataFrame(benchmarks, columns=['framework', 'device', 'benchmark', 'execution', 'memory', 'heap_free', 'heap_alloc', 'heap_size', 'mem_data_low', 'mem_data_med', 'mem_data_high', 'energy', 'foreground_time', 'cpu_time'])

unique = lambda x: list(set(x))

<h1>PARAMETER CHOSE</h1>

In [3]:


def getValue(df, benchmark, framework):
    try:
        filt = (df['benchmark'] == benchmark) & (df['framework'] == framework)
        filtered = df.loc[ filt ]
        return filtered["foreground_time"].iloc[0]
    except:
        return np.NaN


def getDeviceDF(device, complete_df):
    device_df = complete_df.loc[ complete_df['device'] == device ]
    frameworks = unique(device_df["framework"].tolist())
    bench_names = unique(map(lambda x: x.split("-")[0], device_df["benchmark"].tolist()))
    executions = []
    for bench_name in bench_names:
        bench_df = device_df.loc[ device_df["benchmark"].str.contains(bench_name, na=False) ]
        benchmarks = sorted(unique(bench_df["benchmark"].tolist()), key=lambda x: int(x.split("-")[1]))
        for benchmark in benchmarks:
            frameworks_result = map(lambda x: getValue(device_df, benchmark, x), frameworks)
            executions.append([benchmark, *frameworks_result])
    return pd.DataFrame(executions, columns=["benchmark", *frameworks])

In [4]:
num_experiences = 1
experiment_relative_path = 'experiment-results-bench-test'
benchmarks = ext.readExperimentResult(experiment_relative_path, num_experiences , readExecutionData)
df = getDataFrame(benchmarks)

,framework,device,benchmark,execution,memory,heap_free,heap_alloc,heap_size,mem_data_low,mem_data_med,mem_data_high,energy,foreground_time,cpu_time
0,C#,SM-G781B,binarytree-19,1,239.302,7.928,29.607,37.536,233.0,233.0,234.0,22.750200,11.387,25.436
1,C#,SM-G781B,binarytree-20,1,625.507,7.885,29.967,37.853,611.0,611.0,611.0,62.825220,23.489,52.460
2,C#,SM-G781B,binarytree-21,1,715.912,7.946,29.441,37.388,699.0,699.0,699.0,126.554400,49.651,108.354
3,C#,SM-G781B,fannkuch-10,1,91.214,8.628,29.073,37.702,89.0,89.0,89.0,0.705484,0.527,0.341
4,C#,SM-G781B,fannkuch-11,1,97.629,7.918,29.362,37.281,95.0,95.0,95.0,7.506475,1.644,7.292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,REACT,SM-G781B,nbody-15000000,1,112.126,8.143,15.939,24.083,109.0,109.0,109.0,42.657732,27.098,29.589
188,REACT,SM-G781B,nbody-22500000,1,112.344,7.734,15.547,23.282,109.0,109.0,110.0,64.057932,40.499,44.417
189,REACT,SM-G781B,nbody-45000000,1,117.241,7.602,15.623,23.226,114.0,114.0,114.0,126.974520,80.184,88.156
190,REACT,SM-G781B,regex-2500000,1,2609.617,49.045,15.984,65.030,1.9,2.2,2.5,204.145920,14.885,146.154


In [5]:
getDeviceDF("SM-G781B", df)

,benchmark,FLUTTER,IONIC,C#,JAVA,REACT
0,fannkuch-10,0.552,0.862,0.527,6.260,1.570
1,fannkuch-11,2.613,4.154,1.644,13.476,12.702
2,fannkuch-12,43.650,50.226,15.600,110.904,NaN
3,nbody-10000000,2.062,2.688,3.296,6.922,18.505
4,nbody-15000000,2.689,3.570,4.857,7.629,27.098
5,nbody-22500000,3.731,4.954,6.811,8.502,40.499
6,nbody-45000000,6.857,9.405,13.055,11.415,80.184
7,nbody-80000000,11.702,15.748,22.638,15.872,NaN
8,nbody-120000000,17.276,23.282,33.394,20.951,NaN
9,nbody-180000000,25.570,35.688,50.062,28.538,NaN
